#### Author: MengChen Chung

## Spin up cluster

In [ ]:
from dask_yarn import YarnCluster
from dask.distributed import Client

In [ ]:
# Create a cluster where each worker has 1 cores and 4 GiB of memory:
cluster = YarnCluster(environment="/home/hadoop/environment.tar.gz",
                      worker_vcores = 1,
                      worker_memory = "4GiB"
                      )

# Scale cluster out to 8 such workers:
cluster.scale(8)

# Connect to the cluster (before proceeding, you should wait for workers to be registered by the dask scheduler, as below):
client = Client(cluster)

In [ ]:
client

## Process geodata

In [161]:
# import modules
import dask
from dask import delayed
import rioxarray
import geopandas as gpd


def get_average_vegetation(tif_file, shape_file, state_idx):
    # read in tif file
    with rioxarray.open_rasterio(tif_file,
                                 chunks={'band': 1, 'x': 1024, 'y': 1024}) as xds:
        # clip the raster data based on the boundary
        # handle 'MultiPolygon' and 'Polygon' in shape file
        if shape_file.geometry[state_idx] != 'MultiPolygon':
            clipped = xds.rio.clip([shape_file.geometry[state_idx]])
        else:
            clipped = xds.rio.clip(shape_file.geometry[state_idx])
        # stack DataArray to be 1D
        clipped = clipped.squeeze().stack(z=("x", "y"))
        # filter out invalid index (-3000) then calculate the mean
        clipped = clipped[clipped!=-3000].mean()
    return clipped

In [162]:
# read in shape file
shape_file = gpd.read_file("cb_2018_us_state_500k/cb_2018_us_state_500k.shp")
# filter out non-state in the shape file
state_idx_list = [i for i in range(0, 56) if i not in (13,27,36,37,38,42,44,45)]
# creat NDVI and EVI lists
NDVI_file_list = ['NDVI/MOD13A1.006__500m_16_days_NDVI_doy2018{0:03d}_aid0001.tif'.format(i) for i in range(1,365,16)]
EVI_file_list = ['EVI/MOD13A1.006__500m_16_days_EVI_doy2018{0:03d}_aid0001.tif'.format(i) for i in range(1,365,16)]

In [163]:
NDVI_dic={} # dictionary for storing each state's avg index
for i, tif_file in enumerate(NDVI_file_list):
    for state_idx in state_idx_list:
        clipped = get_average_vegetation(tif_file, shape_file, state_idx)
        # first time we create keys and values
        # other times we update the dictionary
        if i == 0:
            NDVI_dic[state_idx] = clipped
        else:
            NDVI_dic[state_idx] += clipped
NDVI_dic

{0: Delayed('add-02492bbc1af26bb342703b01ba6621dd'),
 1: Delayed('add-b2f33e0d1bc0b80a157ab0af26b340d4'),
 2: Delayed('add-8d0fdea1661775733fee6225941a980b'),
 3: Delayed('add-43dc0331dbaf91f28980e9d1fdad25c3'),
 4: Delayed('add-bc731768a1dde4645b97900a06400aae'),
 5: Delayed('add-ea9bf5271a31772789fead1e0bd8c39c'),
 6: Delayed('add-b3f5893662c9c0e5099df05e50b5fe8d'),
 7: Delayed('add-8760d410aae3231d6ac9eface6557f3d'),
 8: Delayed('add-d455c2850a8a1e0a9b06347f410e3042'),
 9: Delayed('add-884ef909bfa887440153572edeceb5a2'),
 10: Delayed('add-ce82bb6d5738821b64e7b314468839ff'),
 11: Delayed('add-a27e78459c4abd14b4fc07041b2bdebc'),
 12: Delayed('add-b60775d5f3d47fe151c357676b617fe8'),
 14: Delayed('add-67f28327c5f35391f40cc45aa8dd4385'),
 15: Delayed('add-ebba09c8f5a8ffb5caa825930c0b7350'),
 16: Delayed('add-638e08a21a70c8ff61239119a0dc338e'),
 17: Delayed('add-02d82c6b13649842afff8d61c5065e18'),
 18: Delayed('add-d1ea00e96c7f12230e92ecb250fe3b30'),
 19: Delayed('add-54d8fa1baf4bcc7acb91

In [178]:
# compute all the values in NDVI_dic and update the dictionary
NDVI_dic = {k: v.compute() for k, v in NDVI_dic.items()}
NDVI_dic

Counter({0: <xarray.DataArray ()>
         array(154382.25880277)
         Coordinates:
             band         int64 1
             spatial_ref  int64 0,
         1: <xarray.DataArray ()>
         array(151986.26649021)
         Coordinates:
             band         int64 1
             spatial_ref  int64 0,
         2: <xarray.DataArray ()>
         array(112027.92206453)
         Coordinates:
             band         int64 1
             spatial_ref  int64 0,
         3: <xarray.DataArray ()>
         array(152561.5162586)
         Coordinates:
             band         int64 1
             spatial_ref  int64 0,
         4: <xarray.DataArray ()>
         array(145094.55589142)
         Coordinates:
             band         int64 1
             spatial_ref  int64 0,
         5: <xarray.DataArray ()>
         array(145832.16930434)
         Coordinates:
             band         int64 1
             spatial_ref  int64 0,
         6: <xarray.DataArray ()>
         array(112133.009

In [171]:
EVI_dic={} # dictionary for storing each state's avg index
for i, tif_file in enumerate(EVI_file_list):
    for state_idx in state_idx_list:
        clipped = get_average_vegetation(tif_file, shape_file, state_idx)
        # first time we create keys and values
        # other times we update the dictionary
        if i == 0:
            EVI_dic[state_idx] = clipped
        else:
            EVI_dic[state_idx] += clipped
EVI_dic

{0: Delayed('add-2e42682ddbf16d6ef37462ce2a6bbccb'),
 1: Delayed('add-88742ae9da6e787df7c7e6ec194b537b'),
 2: Delayed('add-09a9901c3bce4b2f6e35ed5acf18a928'),
 3: Delayed('add-1550d0736a6e7766f560ce64870d04ef'),
 4: Delayed('add-3dcad44208a863bb184454cdd87c185b'),
 5: Delayed('add-8d2a1b6093bf2577bf9841b6b5f8df97'),
 6: Delayed('add-6ebeda50352fe9e8535489631f707ff8'),
 7: Delayed('add-9082935d3de2b4b9bc3d2ea8bd2eacde'),
 8: Delayed('add-662053bfcea2223864dc42df33def015'),
 9: Delayed('add-d9b71bc8387888993b56d943f432cef7'),
 10: Delayed('add-7f73efc00eea7a738482d3027300eaf3'),
 11: Delayed('add-a3b06a2e52d355a1e00e81d6771f6317'),
 12: Delayed('add-f1ffc538ed75adbd2dae9111d963d994'),
 14: Delayed('add-4dc9f0cd34d497b640fd9003775e845c'),
 15: Delayed('add-f0d1a6c80807349f074cab6f4c9552f2'),
 16: Delayed('add-9ee97f4a869a22e932c1cfcddcd32240'),
 17: Delayed('add-92c25c27ea222735a1ef92f739595c1a'),
 18: Delayed('add-3990a9c10e37a0186e3f221b26290526'),
 19: Delayed('add-df898139dadb9f4e3d83

In [176]:
# compute all the values in EVI_dic and update the dictionary
EVI_dic = {k: v.compute() for k, v in EVI_dic.items()}
EVI_dic

Counter({0: <xarray.DataArray ()>
         array(87154.05748272)
         Coordinates:
             band         int64 1
             spatial_ref  int64 0,
         1: <xarray.DataArray ()>
         array(86581.12164828)
         Coordinates:
             band         int64 1
             spatial_ref  int64 0,
         2: <xarray.DataArray ()>
         array(66138.50788721)
         Coordinates:
             band         int64 1
             spatial_ref  int64 0,
         3: <xarray.DataArray ()>
         array(88907.74063775)
         Coordinates:
             band         int64 1
             spatial_ref  int64 0,
         4: <xarray.DataArray ()>
         array(86548.04387523)
         Coordinates:
             band         int64 1
             spatial_ref  int64 0,
         5: <xarray.DataArray ()>
         array(81485.85106271)
         Coordinates:
             band         int64 1
             spatial_ref  int64 0,
         6: <xarray.DataArray ()>
         array(68628.80720701)

In [113]:
# save the indexes
import pickle
with open('NDVI.pickle', 'wb') as outputfile:
    pickle.dump(NDVI_dic, outputfile)

with open('NDVI.pickle', 'rb') as inputfile:
    print(pickle.load(inputfile))

with open('EVI.pickle', 'wb') as outputfile:
    pickle.dump(EVI_dic, outputfile)

with open('EVI.pickle', 'rb') as inputfile:
    print(pickle.load(inputfile))

Counter({17: <xarray.DataArray ()>
array(90926.25451864)
Coordinates:
    band         int64 1
    spatial_ref  int64 0, 3: <xarray.DataArray ()>
array(88907.74063775)
Coordinates:
    band         int64 1
    spatial_ref  int64 0, 23: <xarray.DataArray ()>
array(88682.96506826)
Coordinates:
    band         int64 1
    spatial_ref  int64 0, 47: <xarray.DataArray ()>
array(88213.38140506)
Coordinates:
    band         int64 1
    spatial_ref  int64 0, 0: <xarray.DataArray ()>
array(87154.05748272)
Coordinates:
    band         int64 1
    spatial_ref  int64 0, 1: <xarray.DataArray ()>
array(86581.12164828)
Coordinates:
    band         int64 1
    spatial_ref  int64 0, 4: <xarray.DataArray ()>
array(86548.04387523)
Coordinates:
    band         int64 1
    spatial_ref  int64 0, 18: <xarray.DataArray ()>
array(86096.72367421)
Coordinates:
    band         int64 1
    spatial_ref  int64 0, 33: <xarray.DataArray ()>
array(85109.8368597)
Coordinates:
    band         int64 1
    spatial_re

In [179]:
# convert indexes dictionary into list for adding to dataframe, also do average here
NDVI_list = [i[1].values.item(0)/23 for i in list(NDVI_dic.items())]
EVI_list = [i[1].values.item(0)/23 for i in list(EVI_dic.items())]

In [181]:
NDVI_list

[6712.272121859755,
 6608.098543052684,
 4870.779220196754,
 6633.109402547956,
 6308.458951800929,
 6340.529100188768,
 4875.348258060799,
 6225.011381415092,
 3420.6138391513155,
 6295.864122423586,
 3732.586165090251,
 4951.431141213172,
 2478.390199148947,
 3393.9714621308117,
 4248.459263274486,
 3824.7183559561126,
 7010.7224828296075,
 6752.751685297581,
 5924.2900831230745,
 5699.448140817931,
 2992.0327143919417,
 2230.3446075707884,
 6474.724662254448,
 2228.2300088872544,
 5407.943453822171,
 4197.5793831600195,
 1779.6861688601841,
 4823.127260854282,
 5409.342518895755,
 2882.6790647734747,
 4276.579665612414,
 6734.164416674608,
 5982.546318249176,
 2334.3735755864654,
 5845.7876042789785,
 5989.6723441561535,
 5606.616920457251,
 5760.423102032725,
 6216.009700690811,
 6247.045525235761,
 5302.50169969118,
 4713.031151524126,
 4520.762121372321,
 2827.0345584978377,
 6199.313052186504,
 5089.855328467723,
 3912.0738895734867,
 6364.925004259647]

In [180]:
EVI_list

[3789.306847074834,
 3764.3965934032685,
 2875.5872994441097,
 3865.553940771869,
 3762.9584293577013,
 3542.863089683165,
 2983.861182913645,
 3542.7201400620766,
 1854.7473409734393,
 3518.862688871018,
 2369.4096496355296,
 2604.5536262619585,
 1449.3875222267905,
 2051.5272381646673,
 2505.3160688520343,
 2023.8174308875048,
 3953.3154138538716,
 3743.335811922257,
 3594.4734314579205,
 3519.546443784024,
 1696.0180296188523,
 1334.5004944407428,
 3855.781089924462,
 1285.9434461190333,
 3350.0859506955953,
 2600.1590171419275,
 1069.7481873453048,
 3074.611484122383,
 3448.90220486455,
 1569.000888970618,
 2796.502761091818,
 3700.4276895523312,
 3624.8118833471813,
 1347.1310642089527,
 3303.4959923548613,
 3556.598326112741,
 3402.5058596310782,
 3521.228727217953,
 3511.4646884843014,
 3835.3644089156574,
 3341.057210418843,
 2933.558317882911,
 2304.092297137345,
 1755.5811698317905,
 3530.9816698596755,
 3259.479447412369,
 2385.2664577004452,
 3699.8172718881933]

## Gather physical and mental data

In [132]:
# read in physical and mental score
import pandas as pd
physical_score = pd.read_csv('physical_score.csv', index_col=0)
mental_score = pd.read_csv('mental_score.csv', index_col=0)

In [182]:
physical_score['NDVI'] = NDVI_list
physical_score['EVI'] = EVI_list
mental_score['NDVI'] = NDVI_list
mental_score['EVI'] = EVI_list

In [184]:
physical_score

,State,Sum,NDVI,EVI
0,MS,590.1,6712.272122,3789.306847
1,NC,455.4,6608.098543,3764.396593
2,OK,577.1,4870.779220,2875.587299
3,VA,420.8,6633.109403,3865.553941
4,WV,546.0,6308.458952,3762.958429
5,LA,540.1,6340.529100,3542.863090
6,MI,496.4,4875.348258,2983.861183
7,MA,367.6,6225.011381,3542.720140
8,ID,443.2,3420.613839,1854.747341
9,FL,400.4,6295.864122,3518.862689


In [183]:
mental_score

,State,Sum,NDVI,EVI
0,MS,4018.8,6712.272122,3789.306847
1,NC,28536.7,6608.098543,3764.396593
2,OK,8683.0,4870.779220,2875.587299
3,VA,17214.0,6633.109403,3865.553941
4,WV,11564.2,6308.458952,3762.958429
5,LA,14226.1,6340.529100,3542.863090
6,MI,32346.0,4875.348258,2983.861183
7,MA,28045.9,6225.011381,3542.720140
8,ID,2833.9,3420.613839,1854.747341
9,FL,62372.2,6295.864122,3518.862689


## Modeling

In [146]:
import statsmodels.api as sm

In [185]:
# formula = 'vegetation access ~ physical health compound score'

X_physical = physical_score[['NDVI', 'EVI']]
y_physical = physical_score['Sum']
model_physical = sm.OLS(y_physical, X_physical)
reg_physical = model_physical.fit()
print(reg_physical.summary())

                                 OLS Regression Results                                
Dep. Variable:                    Sum   R-squared (uncentered):                   0.934
Model:                            OLS   Adj. R-squared (uncentered):              0.932
Method:                 Least Squares   F-statistic:                              328.0
Date:                Thu, 03 Jun 2021   Prob (F-statistic):                    5.98e-28
Time:                        18:44:04   Log-Likelihood:                         -296.61
No. Observations:                  48   AIC:                                      597.2
Df Residuals:                      46   BIC:                                      601.0
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [186]:
# formula = 'vegetation access ~ mental health compound score'

X_mental = mental_score[['NDVI', 'EVI']]
y_mental = mental_score['Sum']
model_mental = sm.OLS(y_mental, X_mental)
reg_mental = model_mental.fit()
print(reg_mental.summary())

                                 OLS Regression Results                                
Dep. Variable:                    Sum   R-squared (uncentered):                   0.547
Model:                            OLS   Adj. R-squared (uncentered):              0.527
Method:                 Least Squares   F-statistic:                              27.74
Date:                Thu, 03 Jun 2021   Prob (F-statistic):                    1.25e-08
Time:                        18:44:10   Log-Likelihood:                         -532.31
No. Observations:                  48   AIC:                                      1069.
Df Residuals:                      46   BIC:                                      1072.
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------